In [26]:
import pandas as pd
import numpy as np
from doe_gensplit.doe import x2fx
from doe_gensplit.utils import obs_var

model = np.array([
    [0, 0, 0, 0, 0, 0],
    [1, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0],
    [0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 1],
    [1, 1, 0, 0, 0, 0],
    [1, 0, 1, 0, 0, 0],
    [1, 0, 0, 1, 0, 0],
    [1, 0, 0, 0, 1, 0],
    [1, 0, 0, 0, 0, 1],
    [0, 1, 1, 0, 0, 0],
    [0, 1, 0, 1, 0, 0],
    [0, 1, 0, 0, 1, 0],
    [0, 1, 0, 0, 0, 1],
    [0, 0, 1, 1, 0, 0],
    [0, 0, 1, 0, 1, 0],
    [0, 0, 1, 0, 0, 1],
    [0, 0, 0, 1, 1, 0],
    [0, 0, 0, 1, 0, 1],
    [0, 0, 0, 0, 1, 1],
])

tg_ex5 = pd.read_excel('../assets/ex5_trinca_gilmour.xlsx').to_numpy()
Xref = x2fx(tg_ex5, model)

In [27]:
from doe_gensplit.doe import doe
from doe_gensplit.validation import validate_design, validate_model

# The factor columns
factors = np.array([
    [2, 1],         # C:   level 2; continuous
    [2, 1],         # B:   level 1; continuous
    [1, 1],         # A:   level 1; continuous
    [0, 1],         # LOC: level 1; continuous
    [0, 1],         # Capacity: level 1; continuous
    [0, 1],         # RPM: level 1; continuous
])

# Plot sizes
plot_sizes = np.array([2, 2, 8], dtype=np.int64)

# Compute V-matrix (observation correlation matrix)
V = obs_var(plot_sizes)

# Validate the model (assertion)
validate_model(model, factors)

# # Create optimal design
# best_Y, metrics = doe(model, plot_sizes, factors, n_tries=10000)

# # Validation
# best_X = x2fx(best_Y, model)
# det_val = np.linalg.det(best_X.T @ np.linalg.solve(V, best_X))

# # Validation and output
# print()
# print('===============================')
# print('Validation')
# validate_design(best_Y, model, factors, plot_sizes)
# print('-------------------------------')
# print('Determinant:', np.max(metrics)**(1/len(model)), det_val**(1/len(model)))
# pd.DataFrame(best_Y).to_csv('../out/ex5_paper.csv', index=False)

best_Y = pd.read_csv('../out/ex5_paper.csv').to_numpy()
best_X = x2fx(best_Y, model)

In [37]:
# # Create optimal design
# best_Y, metrics = doe(model, plot_sizes, factors, ratios=np.array([1, 10, 10], dtype=np.float64), n_tries=10000)

# # Validation
# best_X = x2fx(best_Y, model)
# V = obs_var(plot_sizes, ratios=np.array([1, 10, 10], dtype=np.float64))
# det_val = np.linalg.det(best_X.T @ np.linalg.solve(V, best_X))

# # Validation and output
# print()
# print('===============================')
# print('Validation')
# validate_design(best_Y, model, factors, plot_sizes)
# print('-------------------------------')
# print('Determinant:', np.max(metrics)**(1/len(model)), det_val**(1/len(model)))
# pd.DataFrame(best_Y).to_csv('../out/ex5_2_paper.csv', index=False)

best_Y_2 = pd.read_csv('../out/ex5_2_paper.csv').to_numpy()
best_X_2 = x2fx(best_Y_2, model)

In [43]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(1, 2, subplot_titles=['$r_{2,3} = 1$', '$r_{2,3} = 10$'], horizontal_spacing=0.05, vertical_spacing=0.1)

##################################################################
efficiencies = np.zeros((3, 3))
for i, eta_3 in enumerate([0.1, 1, 10]):
    for j, eta_2 in enumerate([0.1, 1, 10]):
        V = obs_var(np.array([2, 2, 8]), ratios=np.array([1, eta_2, eta_3], dtype=np.float64))

        Mref = Xref.T @ np.linalg.solve(V, Xref)
        detref = np.linalg.det(Mref) ** (1/len(model))

        M = best_X.T @ np.linalg.solve(V, best_X)
        det = np.linalg.det(M) ** (1/len(model))
        efficiencies[i, j] = det / detref

fig.add_trace(go.Heatmap(
    z=efficiencies*100,
    x=['0.1', '1', '10'],
    y=['0.1', '1', '10'],
    coloraxis='coloraxis',
    text=efficiencies*100,
    texttemplate='%{text:.2f}%',
    textfont={'size': 8}
), row=1, col=1)

#################################################

efficiencies = np.zeros((3, 3))
for i, eta_3 in enumerate([0.1, 1, 10]):
    for j, eta_2 in enumerate([0.1, 1, 10]):
        V = obs_var(np.array([2, 2, 8]), ratios=np.array([1, eta_2, eta_3], dtype=np.float64))

        Mref = Xref.T @ np.linalg.solve(V, Xref)
        detref = np.linalg.det(Mref) ** (1/len(model))

        M = best_X_2.T @ np.linalg.solve(V, best_X_2)
        det = np.linalg.det(M) ** (1/len(model))
        efficiencies[i, j] = det / detref

fig.add_trace(go.Heatmap(
    z=efficiencies*100,
    x=['0.1', '1', '10'],
    y=['0.1', '1', '10'],
    coloraxis='coloraxis',
    text=efficiencies*100,
    texttemplate='%{text:.2f}%',
    textfont={'size': 8}
), row=1, col=2)

#################################################

fig.update_xaxes(title='$\eta_2$')
fig.update_layout(
    coloraxis={'colorscale': 'viridis'},
    yaxis_title='$\eta_3$',
    height=300,
)
fig.show()
fig.write_image('../figures/ex5_comp.png')